In [1]:
import sys 
sys.path.append('../..')

import math
import numpy as np
import torch

import gpytorch
from gpytorch.means import ZeroMean
from gpytorch.kernels import RBFKernel, PeriodicKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import VariationalELBO, PredictiveLogLikelihood

from horseshoe_gp.src.structural_sgp import VariationalGP, StructuralSparseGP, \
TrivialSelector, SpikeAndSlabSelector, SpikeAndSlabSelectorV2, HorseshoeSelector
from horseshoe_gp.src.mean_field_hs import MeanFieldHorseshoe, VariatioalHorseshoe

import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

from scipy.optimize import minimize, Bounds

boston = load_boston()
X_train, X_test, y_train, y_test = train_test_split(
    boston['data'],
    boston['target'],
    test_size=0.2,
    random_state=2020)

#I am not sure scaling per train and test is appropriate
X_train = StandardScaler().fit_transform(X_train)
X_test = StandardScaler().fit_transform(X_test)
y_train = StandardScaler().fit_transform(np.expand_dims(y_train, axis=1))
y_test = StandardScaler().fit_transform(np.expand_dims(y_test, axis=1))

#set up kernels
n_kernels = 5
means = [ZeroMean()] * n_kernels
kernels = [RBFKernel()] * n_kernels

n_inducing = 10
inducing_points = torch.linspace(0, 1, n_inducing)

# GP for each kernel
gps = []
for mean, kernel in zip(means, kernels):
    gp = VariationalGP(mean, kernel, inducing_points)
    gps.append(gp)
    
norm = torch.distributions.normal.Normal(0, 1)

SVM regression on Boston housing dataset

In [2]:
def svm_reg_evaluate(params):
    model = svm.SVR(epsilon=(-10*params[0]).exp(), #log_eps
                   gamma=(-10*param[1]).exp(), #log_gamma
                   C=10000*params[2],
                   kernel='rbf')
    model.fit(X_train, Y_train)
    return mean_squared_error(model.predict(X_test, Y_test), squared = True)

num_param = 3
num_start_pnt = 5
num_optimize = 10

#log_eps, log_gamma, C
lb = np.array([0, 0, 0])
ub = np.array([1, 1, 1])

scope = Bounds(lb, ub)

In [15]:
def UCB(x, sur_model, kappa = 2.576):
    mean, std = sur_model(x)
    return mean + kappa * std

def EI(x, sur_model, y_max):
    mean, std = sur_model(x)
    a = (mean - ymax - x)
    z = a / std
    return a * norm.cdf(z) + std * norm.pdf(z)

def POI(x, sur_model, y_max):
    mean, std = sur_model(x)
    z = (mean - y_max - x)/std
    return norm.cdf(z)

def acq_max(bounds, sur_model, y_max, acq_fun, n_warmup = 10000, iteration = 10):
    x_tries = torch.empty(n_warmup, bounds.shape[0])._rand() * (bounds[:, 1] - bounds[:, 0]) + bounds[:, 0]
    ys = acq_fun(x_tries, sur_model, y_max, kappa)
    x_max = x_tries[ys.argmax()]
    max_acq = ys.max()
    
    for iterate in range(iteration):
        locs = torch.empty(bounds.shape[0])._rand() * (bounds[:, 1] - bounds[:, 0]) + bounds[:, 0]
        res = minimize(lambda x: -acq_fun(x),
                      locs,
                      bounds = bounds,
                      method = "L-BFGS-B")
        
        if not res.success:
            continue

        if max_acq is None or -res.fun[0] >= max_acq:
            x_max = res.x
            max_acq = -res.fun[0]
            
    return torch.clip(x_max, bounds[:, 0], bounds[:, 1])

def bayes_opt(bounds, evaluate, acq_fun = UCB):    
    for num in num_optimize:
        next_pnt = acq_max(bounds, model, y_max, acq_fun = acq_fun)
        model = model.fit(next_pnt)

def experiment_gp_spike_and_slab(bounds):
    selector = SpikeAndSlabSelector(dim=n_kernels, gumbel_temp=.5)
    model = StructuralSparseGP(gps, selector)

    likelihood = GaussianLikelihood()
    elbo = PredictiveLogLikelihood(likelihood, model, num_data=100)
    
    init_pnts = torch.empty(num_start_pnt, bounds.shape[0])._rand() * (bounds[:, 1] - bounds[:, 0]) + bounds[:, 0]
    
    init_targets = []
    for pnt in enumerate(init_pnts):
        init_targets.append(evaluate(pnt))
    init_targets = torch.stack(init_targets, dim = 0)
    
    for ind, pnt in enumerate(init_pnts):
        if ind:
            loss += elbo(model(pnt), evaluate(pnt))
        else:
            loss = elbo(model(pnt), evaluate(pnt))
            
        loss.backward()
    
def experiment_gp_horseshoe(bounds):
    
    

In [1]:
import botorch

In [2]:
[1] * 2

[1, 1]

In [3]:
len(1)

TypeError: object of type 'int' has no len()

In [4]:
isinstance(1, int)

True